This notebook loads skill action data from the skill calculators on the OSRS wiki i.e. https://oldschool.runescape.wiki/w/Calculator:Crafting  
It then correctly parses and normalizes the data and saves the data to the '' tables

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
from pyspark.sql import functions as sf

In [0]:
# Crafting Skill activities
# data is raw data from https://oldschool.runescape.wiki/w/Calculator:Crafting

df_crafting = (
    spark.read.format("json")
    .option("multiline", "true")
    .load("/Volumes/runescape/00_landing/data_sources/skill_actions/crafting.json")
)

# select only columns that we care about and rename them to the correct column names
df_crafting = df_crafting.select(
    sf.col("level").alias("output"),
    sf.col("Input Cost").alias("input"),
    sf.col("# Needed").alias("xp"),
).withColumn("skill", sf.lit("crafting"))

# TODO decide if I should append "crafting" to this data as mthod

# TODO create function to call that takes the above df and parses it using code below

# need to split the input column
df = df_crafting.select("*", sf.split(df_crafting.input, "[\n]").alias("split_input"))


In [0]:
df.display()


In [0]:
df2 = df.select('output', sf.explode('split_input').alias("input"), 'skill')
df2.display()

In [0]:
# split string by the multiplcation symbol '×'
df3 = df2.select('output', sf.split(df2.input, '[×]').alias("input"), 'skill')
df3.display()

In [0]:
df4 = df3.select('output', sf.get(df3.input,0).cast("double").alias("input_qty"), sf.get(df3.input,1).alias("input_name"), 'skill')
df4.display()